In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from sklearn.utils import shuffle
import pandas as pd

In [2]:
path =  ['/home/ayon/Database/IsolatedTrain/', '/home/ayon/Database/IsolatedTest/']

In [3]:
global params
params = {'path' : path,
          'batch_size' : 50,
          'input_size': (32,32), 
          'lrn_rate': 0.0001, 
          'epochs': 200}

In [4]:
def get_file_paths(path):
    path_train, path_test = path
    path_train = shuffle(sorted([os.path.join(root, file)  for root, dirs, files in os.walk(path_train) for file in files]))
    path_test = shuffle(sorted([os.path.join(root, file)  for root, dirs, files in os.walk(path_test) for file in files]))
    return path_train, path_test

In [5]:
def load_data(path):
    img = cv2.imread(path, 0)
    if img is None: return (False, None)
    img = cv2.resize(cv2.threshold(img, 240, 1, cv2.THRESH_BINARY)[1], 
                     params['input_size'], 
                     cv2.INTER_AREA)
    return np.expand_dims(img, -1)

In [6]:
def load_label(im_path):
    return int(im_path.split('/')[5])-1

In [7]:
def to_one_hot(label_data, num_class):
    num_sample = np.shape(label_data)[0]
    temp = np.zeros([num_sample, num_class])
    temp[np.arange(num_sample),label_data] = 1
    return temp

In [8]:
def Network(Input): #input : [Batch_size, 32, 32, 1]
    with tf.name_scope("Network"):
        conv1 = tf.layers.conv2d(Input, filters = 32, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv1')
        conv2 = tf.layers.conv2d(conv1, filters = 32, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv2')        
        conv3 = tf.layers.conv2d(conv2, filters = 32, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv3')
        pool1 = tf.layers.max_pooling2d(conv3, pool_size = 2, strides = 2, name = 'pool1')
        conv4 = tf.layers.conv2d(pool1, filters = 64, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv4')
        conv5 = tf.layers.conv2d(conv4, filters = 64, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv5')
        conv6 = tf.layers.conv2d(conv5, filters = 64, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv6')
        pool2 = tf.layers.max_pooling2d(conv6, pool_size = 2, strides = 2, name = 'pool2')
        conv7 = tf.layers.conv2d(pool2, filters = 128, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv7')
        conv8 = tf.layers.conv2d(conv7, filters = 128, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv8')
        conv9 = tf.layers.conv2d(conv8, filters = 128, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv9')
        conv10 = tf.layers.conv2d(conv9, filters = 128, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv10')
        conv11 = tf.layers.conv2d(conv10, filters = 128, kernel_size = 3, strides = 1,  padding='same' ,activation = tf.nn.relu, name = 'conv11')
      
        flat = tf.contrib.layers.flatten(tf.layers.batch_normalization(conv11))
        fc1 = tf.layers.dense(flat, units = 512, activation = tf.nn.relu, name = 'fc1')
        fc2 = tf.layers.dense(fc1, units = 512, activation = tf.nn.relu, name = 'fc2')   
        fc3 = tf.layers.dense(fc2, units = 171, activation = None, name = 'fc3')
    return fc3

In [9]:
def loss_function(logit, Label):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels = Label), name='Loss')
    return loss

In [10]:
def Accuracy_Evaluate(prediction, Label):
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Label, 1), name='Corr_Pred')
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='Accuracy')
    return correct_pred, accuracy

In [11]:
def classifier():
    
    batchsize = params['batch_size']
    Input = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32, 1])
    Label = tf.placeholder(dtype = tf.float32, shape = [None, 171])
    
    logit = Network(Input)
    
    prediction = tf.nn.softmax(logit)
    
    loss = loss_function(logit, Label)
    
    correct_pred, accuracy = Accuracy_Evaluate(prediction, Label)
    
    
    optimiz = tf.train.AdadeltaOptimizer(params['lrn_rate']).minimize(loss)

    init = tf.global_variables_initializer()
    sess.run(init)
 
    
    tf.summary.scalar('Loss_Value',loss)
    tf.summary.scalar('Accuracy',accuracy)
    
    print('Setting up summary op...')
    summary_op = tf.summary.merge_all()
    
    print('Setting up train summary...')
    train_summary_writer = tf.summary.FileWriter('./log_dir/train', sess.graph)
    
    print('Setting up test summary....')
    test_summary_writer = tf.summary.FileWriter('./log_dir/test', sess.graph)
    t_summary_writer = tf.summary.FileWriter('./log_dir/t', sess.graph)
       
    train_paths, test_paths = get_file_paths(params['path'])
    dataset_size = len(train_paths)
    n_batches = dataset_size//batchsize
    
    itr = 1
    for epoch in range(params['epochs']):
         
        for idx in range(n_batches): 
            
            batch_paths = train_paths[idx * batchsize: (idx+1) * batchsize]
            batch_data = np.array([load_data(path) for path in batch_paths])
            batch_label = to_one_hot(np.array([load_label(path) for path in batch_paths]), 171)
            
            feed_dict = {Input : batch_data , Label : batch_label}
            
            _, train_loss, train_accuracy, train_summary_str = sess.run([optimiz, loss, accuracy, summary_op] , feed_dict )
            train_summary_writer.add_summary(train_summary_str, itr)
            itr = itr + 1
            
            if idx%10 == 0:
                
                print ('epoch : '+str(epoch)+' step : '+str(idx) + ' train_loss : '+str(train_loss) +
                        ' train_accuracy : '+str(train_accuracy) 
                        )
                
            if idx%100 == 0:
                test_batch_paths = np.array(test_paths)[np.random.choice(len(test_paths), batchsize)]
                test_batch_data = np.array([load_data(path) for path in test_batch_paths])
                test_batch_label = to_one_hot(np.array([load_label(path) for path in test_batch_paths]), 171)
                feed_dict = {Input: test_batch_data, Label: test_batch_label}

                test_accuracy, test_loss, test_summary_str = sess.run([accuracy, loss, summary_op], 
                                               feed_dict=feed_dict)
                test_summary_writer.add_summary(test_summary_str, itr)
                
                print('epoch : ' +str(epoch)+' step : '+str(idx)+' test_loss : '+str(test_loss)+' test_accuracy : '+str(test_accuracy))
        
        if epoch%40 == 0:
            for idx in range(n_batches):
                t_batch_paths = test_paths[:]
                t_batch_data = np.array([load_data(path) for path in t_batch_paths])
                t_batch_label = np.array([load_label(path) for path in t_batch_paths])
                t_accuracy, t_loss, t_summary_str = sess.run([accuracy, loss, summary_op], 
                                               feed_dict=feed_dict)
                t_summary_writer.add_summary(t_summary_str, itr)
                
                print('epoch : ' +str(epoch)+' step : '+str(idx)+' t_loss : '+str(t_loss)+' t_accuracy : '+str(t_accuracy))

In [12]:
from tensorflow.python.framework import ops

In [ ]:
ops.reset_default_graph()
global sess

config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()

classifier()